In [52]:
import numpy as np
import pandas as pd
import random
import time
import math
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import init
from dateutil import parser
from pathlib import Path
import json 
import shutil
import logging
logger = logging.getLogger(str(os.getpid()))

from wattile.data_reading import read_dataset_from_file
from wattile.buildings_processing import correct_predictor_columns, correct_timestamps, resample_or_rolling_stats, timelag_predictors, timelag_predictors_target, roll_predictors_target
from wattile.time_processing import add_processed_time_columns
PROJECT_DIRECTORY = Path().resolve().parent.parent

# ------------------------------------------------------
# reading configs
# ------------------------------------------------------

In [53]:
"""
For this example, we will be using the default configs.
Check out the docs for an explaination of each config.
"""
##################################################################################
# choose the configs file to use as an input
##################################################################################
# main configs file
with open(PROJECT_DIRECTORY / "wattile" / "configs" / "configs.json", "r") as f:
    configs = json.load(f)
##################################################################################
# code testing configs file
# with open(PROJECT_DIRECTORY / "tests" / "fixtures" / "test_configs.json", "r") as f:
#     configs = json.load(f)
##################################################################################

exp_dir = PROJECT_DIRECTORY / "notebooks" / "exp_dir"
if exp_dir.exists():
    shutil.rmtree(exp_dir)
exp_dir.mkdir()

configs["data_input"]["exp_dir"] = str(PROJECT_DIRECTORY / exp_dir)
configs["data_input"]["data_dir"] = str(PROJECT_DIRECTORY / "data" / "Synthetic Site")

configs

{'data_input': {'data_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\data\\Synthetic Site',
  'data_config': 'Synthetic Site Config.json',
  'start_time': '2018-01-01T00:00:00-07:00',
  'end_time': '2022-01-01T00:00:00-07:00',
  'predictor_columns': ['Synthetic Weather Station Dew Point Temperature',
   'Synthetic Weather Station Diffuse Horizontal Irradiance',
   'Synthetic Weather Station Direct Normal Irradiance',
   'Synthetic Weather Station Dry Bulb Temperature',
   'Synthetic Weather Station Global Horizontal Irradiance',
   'Synthetic Weather Station Relative Humidity',
   'Synthetic Weather Station Wind Speed'],
  'target_var': 'Synthetic Site Electricity Main Total Power',
  'exp_dir': 'C:\\Users\\JKIM4\\Documents\\GitHub\\intelligentcampus-pred-analytics\\notebooks\\exp_dir'},
 'data_output': {'exp_dir': 'exp_dir',
  'plot_comparison': True,
  'plot_comparison_portion_start': 0.0,
  'plot_comparison_portion_end': 1.0},
 'data_processing': {'fea

# ------------------------------------------------------
# reading data
# ------------------------------------------------------

In [54]:
configs["target_feat_name"] = [configs["data_input"]["target_var"]]
data = read_dataset_from_file(configs)
data

,Synthetic Weather Station Dew Point Temperature,Synthetic Weather Station Diffuse Horizontal Irradiance,Synthetic Weather Station Direct Normal Irradiance,Synthetic Weather Station Dry Bulb Temperature,Synthetic Weather Station Global Horizontal Irradiance,Synthetic Weather Station Relative Humidity,Synthetic Weather Station Wind Speed,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,104.800003
2021-12-01 07:01:00+00:00,15.183906,-1.496226,0.207964,56.174000,-1.841866,17.879999,11.265539,96.650002
2021-12-01 07:02:00+00:00,14.891221,-1.441191,0.457520,55.886002,-1.873579,17.809999,12.777752,96.199997
2021-12-01 07:03:00+00:00,14.836164,-1.371754,0.473117,55.796001,-1.860084,17.820000,12.694983,95.599998
2021-12-01 07:04:00+00:00,14.782966,-1.317349,0.545903,55.723999,-1.843271,17.820000,11.632407,100.650002
...,...,...,...,...,...,...,...,...
2021-12-08 06:55:00+00:00,11.210565,-1.487212,-0.254757,42.285198,-1.814939,24.680000,0.000000,109.664803
2021-12-08 06:56:00+00:00,11.190062,-1.445182,-0.233960,41.997200,-1.852901,24.930000,0.000000,107.002800
2021-12-08 06:57:00+00:00,11.223961,-1.396302,-0.145575,41.669601,-1.841248,25.290001,3.545647,106.480400


# ------------------------------------------------------
# data processing
# ------------------------------------------------------

In [55]:
# assert we have the correct columns and order them
data = correct_predictor_columns(configs, data)

# sort and trim data specified time period
data = correct_timestamps(configs, data)

# Add time-based features
data = add_processed_time_columns(data, configs)

# Add statistics features
data = resample_or_rolling_stats(data, configs)

data

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,HOD_binary_reg_0_min,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-2.588190e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,104.800003
2021-12-01 07:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-3.214395e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,103.650002
2021-12-01 07:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-3.826834e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:45:00+00:00,16.614972,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-4.422887e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,98.349998
2021-12-01 08:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-5.000000e-01,0.0,...,0.0,0.0,0.705556,0.294444,0.0,0.0,0.0,-0.493776,0.869589,102.949997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:00:00+00:00,11.961447,-1.673728,-1.211398,41.388802,-1.909901,25.160000,0.000000,0.998135,-1.608123e-16,0.0,...,0.0,0.0,0.788889,0.211111,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,11.008480,-1.489343,-0.504314,41.646198,-1.936419,24.240000,0.000000,0.997859,-6.540313e-02,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,102.709602
2021-12-08 06:30:00+00:00,10.851442,-1.571301,-0.655083,42.024200,-1.834018,23.760000,0.000000,0.991445,-1.305262e-01,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,105.972198


In [25]:
# if configs["learning_algorithm"]["arch_version"] == "alfa":
#     data = timelag_predictors(data, configs)
# elif configs["learning_algorithm"]["arch_version"] == "bravo":
#     data = timelag_predictors_target(data, configs)
# elif configs["learning_algorithm"]["arch_version"] == "charlie":
#     data = roll_predictors_target(data, configs)
    
# data

# ------------------------------------------------------
# tesing
# ------------------------------------------------------

## for bravo

In [26]:
configs["data_processing"]["input_output_window"][
        "window_width_futurecast"
    ] = "15min"

window_width_futurecast = configs["data_processing"]["input_output_window"][
        "window_width_futurecast"
    ]

window_width_futurecast

'15min'

In [27]:
# reading configuration parameters
lag_interval = configs["data_processing"]["feat_timelag"]["lag_interval"]
lag_count = configs["data_processing"]["feat_timelag"]["lag_count"]
window_width_target = configs["data_processing"]["input_output_window"][
    "window_width_target"
]
resample_interval = configs["data_processing"]["resample_interval"]
initial_num = pd.Timedelta(window_width_target) // pd.Timedelta(resample_interval)
secondary_num = configs["data_processing"]["input_output_window"]["secondary_num"]
decay = configs["data_processing"]["input_output_window"]["decay"]
target_var = configs["data_input"]["target_var"]

#######################################################################
# shift target for futurecast
data[target_var] = data[target_var].shift(freq="-" + window_width_futurecast)
#######################################################################

target = data[target_var]
data = data.drop(target_var, axis=1)
data_orig = data
# Pad the exogenous variables
temp_holder = list()
temp_holder.append(data_orig)
for i in range(1, lag_count + 1):
    shifted = (
        data_orig.shift(freq=i * lag_interval)
        .astype("float32")
        .add_suffix("_lag{}".format(i))
    )
    temp_holder.append(shifted)
temp_holder.reverse()
data = pd.concat(temp_holder, axis=1)

data

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,...,DOW_binary_reg_6_mean,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:15:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:30:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 07:45:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589
2021-12-01 08:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.705556,0.294444,0.0,0.0,0.0,-0.493776,0.869589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 12:00:00+00:00,11.961447,-1.673728,-1.211398,41.388802,-1.909901,25.160000,0.0,0.998135,-1.608123e-16,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-08 12:15:00+00:00,11.008480,-1.489343,-0.504314,41.646198,-1.936419,24.240000,0.0,0.997859,-6.540313e-02,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-08 12:30:00+00:00,10.851442,-1.571301,-0.655083,42.024200,-1.834018,23.760000,0.0,0.991445,-1.305262e-01,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
# Do fine padding for future predictions. Create a new df to preserve memory usage.
local = pd.DataFrame()
for i in range(0, initial_num):
    if i == 0:
        local["{}_lag_{}".format(target_var, i)] = target.shift(i)
    else:
        local["{}_lag_{}".format(target_var, i)] = target.shift(
            freq="-" + (i * lag_interval)
        )

# Do additional coarse padding for future predictions
for i in range(1, secondary_num + 1):
    base = initial_num
    new = int(base + decay * i)
    local["{}_lag_{}".format(target_var, base + i)] = target.shift(
        freq="-" + (new * lag_interval)
    )

data = pd.concat([data, local], axis=1)

# Drop all nans
data = data.dropna(how="any")

data

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,...,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power_lag_0,Synthetic Site Electricity Main Total Power_lag_1,Synthetic Site Electricity Main Total Power_lag_2
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 13:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-0.258819,0.0,...,0.497222,0.502778,0.0,0.0,0.0,-0.493776,0.869589,99.800003,98.900002,101.500000
2021-12-01 13:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-0.321439,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,98.900002,101.500000,98.504036
2021-12-01 13:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-0.382683,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,101.500000,98.504036,103.149178
2021-12-01 13:45:00+00:00,16.614971,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-0.442289,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,98.504036,103.149178,99.069603
2021-12-01 14:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-0.500000,0.0,...,0.455556,0.544444,0.0,0.0,0.0,-0.493776,0.869589,103.149178,99.069603,97.865646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 05:15:00+00:00,15.582798,13.926300,0.738270,48.812000,13.456160,22.980000,11.688332,-0.254602,0.967046,0.0,...,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,111.518997,109.573402,104.668396
2021-12-08 05:30:00+00:00,15.828789,6.462158,0.155972,48.506001,6.076493,24.150000,14.399578,-0.190809,0.981627,0.0,...,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,109.573402,104.668396,102.709602
2021-12-08 05:45:00+00:00,16.137859,1.064227,-0.384731,47.858002,0.590813,24.799999,10.625756,-0.126199,0.992005,0.0,...,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,104.668396,102.709602,105.972198


### data before revision

In [11]:
data

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,...,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power_lag_0,Synthetic Site Electricity Main Total Power_lag_1,Synthetic Site Electricity Main Total Power_lag_2
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 13:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-0.258819,0.0,...,0.497222,0.502778,0.0,0.0,0.0,-0.493776,0.869589,99.349998,99.800003,98.900002
2021-12-01 13:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-0.321439,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,99.800003,98.900002,101.500000
2021-12-01 13:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-0.382683,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,98.900002,101.500000,98.504036
2021-12-01 13:45:00+00:00,16.614971,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-0.442289,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,101.500000,98.504036,103.149178
2021-12-01 14:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-0.500000,0.0,...,0.455556,0.544444,0.0,0.0,0.0,-0.493776,0.869589,98.504036,103.149178,99.069603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 05:30:00+00:00,15.828789,6.462158,0.155972,48.506001,6.076493,24.150000,14.399578,-0.190809,0.981627,0.0,...,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,111.518997,109.573402,104.668396
2021-12-08 05:45:00+00:00,16.137859,1.064227,-0.384731,47.858002,0.590813,24.799999,10.625756,-0.126199,0.992005,0.0,...,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,109.573402,104.668396,102.709602
2021-12-08 06:00:00+00:00,16.623901,-0.810933,-0.707074,47.695999,-1.170910,26.010000,10.484825,-0.061049,0.998135,0.0,...,0.788889,0.211111,0.0,0.0,0.0,-0.385663,0.922640,104.668396,102.709602,105.972198


### data after revision

In [29]:
# data after revision
data

,Synthetic Weather Station Dew Point Temperature_min_lag24,Synthetic Weather Station Diffuse Horizontal Irradiance_min_lag24,Synthetic Weather Station Direct Normal Irradiance_min_lag24,Synthetic Weather Station Dry Bulb Temperature_min_lag24,Synthetic Weather Station Global Horizontal Irradiance_min_lag24,Synthetic Weather Station Relative Humidity_min_lag24,Synthetic Weather Station Wind Speed_min_lag24,sin_HOD_min_lag24,cos_HOD_min_lag24,HOD_binary_reg_0_min_lag24,...,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power_lag_0,Synthetic Site Electricity Main Total Power_lag_1,Synthetic Site Electricity Main Total Power_lag_2
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 13:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-0.258819,0.0,...,0.497222,0.502778,0.0,0.0,0.0,-0.493776,0.869589,99.800003,98.900002,101.500000
2021-12-01 13:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-0.321439,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,98.900002,101.500000,98.504036
2021-12-01 13:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-0.382683,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,101.500000,98.504036,103.149178
2021-12-01 13:45:00+00:00,16.614971,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-0.442289,0.0,...,0.458333,0.541667,0.0,0.0,0.0,-0.493776,0.869589,98.504036,103.149178,99.069603
2021-12-01 14:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-0.500000,0.0,...,0.455556,0.544444,0.0,0.0,0.0,-0.493776,0.869589,103.149178,99.069603,97.865646
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 05:15:00+00:00,15.582798,13.926300,0.738270,48.812000,13.456160,22.980000,11.688332,-0.254602,0.967046,0.0,...,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,111.518997,109.573402,104.668396
2021-12-08 05:30:00+00:00,15.828789,6.462158,0.155972,48.506001,6.076493,24.150000,14.399578,-0.190809,0.981627,0.0,...,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,109.573402,104.668396,102.709602
2021-12-08 05:45:00+00:00,16.137859,1.064227,-0.384731,47.858002,0.590813,24.799999,10.625756,-0.126199,0.992005,0.0,...,0.791667,0.208333,0.0,0.0,0.0,-0.385663,0.922640,104.668396,102.709602,105.972198


## for charlie

In [56]:
configs["data_processing"]["input_output_window"][
        "window_width_futurecast"
    ] = "15min"

window_width_futurecast = configs["data_processing"]["input_output_window"][
        "window_width_futurecast"
    ]

window_width_futurecast

'15min'

In [57]:
data

,Synthetic Weather Station Dew Point Temperature_min,Synthetic Weather Station Diffuse Horizontal Irradiance_min,Synthetic Weather Station Direct Normal Irradiance_min,Synthetic Weather Station Dry Bulb Temperature_min,Synthetic Weather Station Global Horizontal Irradiance_min,Synthetic Weather Station Relative Humidity_min,Synthetic Weather Station Wind Speed_min,sin_HOD_min,cos_HOD_min,HOD_binary_reg_0_min,...,DOW_binary_fuzzy_0_mean,DOW_binary_fuzzy_1_mean,DOW_binary_fuzzy_2_mean,DOW_binary_fuzzy_3_mean,DOW_binary_fuzzy_4_mean,DOW_binary_fuzzy_5_mean,DOW_binary_fuzzy_6_mean,sin_MOY_mean,cos_MOY_mean,Synthetic Site Electricity Main Total Power
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-12-01 07:00:00+00:00,15.413733,-1.525850,-0.005199,56.408001,-1.810825,17.930000,10.457981,0.965926,-2.588190e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,104.800003
2021-12-01 07:15:00+00:00,14.782966,-1.496226,-0.540707,55.112000,-1.873579,17.809999,9.395406,0.946930,-3.214395e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,103.650002
2021-12-01 07:30:00+00:00,15.474850,-1.379412,-1.741706,54.608002,-1.918852,18.690001,8.612455,0.923880,-3.826834e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,98.050003
2021-12-01 07:45:00+00:00,16.614972,-1.642929,-2.147240,53.743999,-2.029242,19.320000,4.503084,0.896873,-4.422887e-01,0.0,...,0.0,0.0,0.708333,0.291667,0.0,0.0,0.0,-0.493776,0.869589,98.349998
2021-12-01 08:00:00+00:00,16.413343,-1.480409,-0.426325,52.807999,-1.958610,21.090000,3.746977,0.866025,-5.000000e-01,0.0,...,0.0,0.0,0.705556,0.294444,0.0,0.0,0.0,-0.493776,0.869589,102.949997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-08 06:00:00+00:00,11.961447,-1.673728,-1.211398,41.388802,-1.909901,25.160000,0.000000,0.998135,-1.608123e-16,0.0,...,0.0,0.0,0.788889,0.211111,0.0,0.0,0.0,-0.385663,0.922640,104.668396
2021-12-08 06:15:00+00:00,11.008480,-1.489343,-0.504314,41.646198,-1.936419,24.240000,0.000000,0.997859,-6.540313e-02,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,102.709602
2021-12-08 06:30:00+00:00,10.851442,-1.571301,-0.655083,42.024200,-1.834018,23.760000,0.000000,0.991445,-1.305262e-01,0.0,...,0.0,0.0,0.750000,0.250000,0.0,0.0,0.0,-0.385663,0.922640,105.972198


In [58]:
# setting configuration parameters
window_width_source = configs["data_processing"]["input_output_window"][
    "window_width_source"
]
###################################################################################
window_width_futurecast = configs["data_processing"]["input_output_window"][
    "window_width_futurecast"
]
###################################################################################
window_width_target = configs["data_processing"]["input_output_window"][
    "window_width_target"
]
resample_interval = configs["data_processing"]["resample_interval"]
target_var = configs["data_input"]["target_var"]

###################################################################################
# shift target for futurecast
data[target_var] = data[target_var].shift(freq="-" + window_width_futurecast)
###################################################################################

# initialize lists
data_predictor = []
data_target = []

# calculate number of rows based on window size defined by time
window_source_size_count = int(
    pd.Timedelta(window_width_source) / pd.Timedelta(resample_interval)
)
window_target_size_count = int(
    pd.Timedelta(window_width_target) / pd.Timedelta(resample_interval)
)

# set aside timeindex
timestamp = data.iloc[
    : -(window_source_size_count + window_target_size_count - 1), :
].index

# create 3D predictor data
data_shifted_predictor = data.iloc[:-window_target_size_count, :]
for window in data_shifted_predictor.rolling(window=window_width_source):
    if window.shape[0] == window_source_size_count:
        data_predictor.append(
            window.values.reshape(
                (1, window_source_size_count, data_shifted_predictor.shape[1])
            )
        )

# create 3D target data
data_shifted_target = data.loc[
    data.index >= data.shift(freq=window_width_source).index[0], :
][target_var]
for window in data_shifted_target.rolling(window=window_width_target):
    if window.shape[0] == window_target_size_count:
        data_target.append(window.values.reshape((1, window_target_size_count, 1)))

# reshape data dimension
data_predictor = np.concatenate(np.array(data_predictor), axis=0)
data_target = np.concatenate(np.array(data_target), axis=0)

# combine 3D predictor and target data into dictionary
data = {}
data["predictor"] = data_predictor
data["target"] = data_target
data["timestamp"] = timestamp

### data before revision

In [42]:
data["predictor"][0:3,:]

array([[[ 1.54137332e+01, -1.52585006e+00, -5.19907009e-03, ...,
         -4.93775550e-01,  8.69589389e-01,  1.04800003e+02],
        [ 1.47829660e+01, -1.49622595e+00, -5.40706813e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  1.03650002e+02],
        [ 1.54748496e+01, -1.37941206e+00, -1.74170601e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  9.80500031e+01],
        ...,
        [ 1.70106054e+01, -1.74843705e+00, -1.92366803e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  9.79499969e+01],
        [ 1.99186356e+01, -1.78446996e+00, -2.07443309e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  1.02000000e+02],
        [ 2.06585697e+01, -1.50733495e+00, -8.31857800e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  9.95000000e+01]],

       [[ 1.47829660e+01, -1.49622595e+00, -5.40706813e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  1.03650002e+02],
        [ 1.54748496e+01, -1.37941206e+00, -1.74170601e+00, ...,
         -4.93775550e-01,  8.69589389e

In [41]:
data["target"][0:3,:]

array([[[103.25      ],
        [ 99.        ],
        [101.69999695]],

       [[ 99.        ],
        [101.69999695],
        [102.05000305]],

       [[101.69999695],
        [102.05000305],
        [103.05000305]]])

### data after revision with window_width_futurecast = 0min

In [50]:
data["predictor"][0:3,:]

array([[[ 1.54137332e+01, -1.52585006e+00, -5.19907009e-03, ...,
         -4.93775550e-01,  8.69589389e-01,  1.04800003e+02],
        [ 1.47829660e+01, -1.49622595e+00, -5.40706813e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  1.03650002e+02],
        [ 1.54748496e+01, -1.37941206e+00, -1.74170601e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  9.80500031e+01],
        ...,
        [ 1.70106054e+01, -1.74843705e+00, -1.92366803e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  9.79499969e+01],
        [ 1.99186356e+01, -1.78446996e+00, -2.07443309e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  1.02000000e+02],
        [ 2.06585697e+01, -1.50733495e+00, -8.31857800e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  9.95000000e+01]],

       [[ 1.47829660e+01, -1.49622595e+00, -5.40706813e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  1.03650002e+02],
        [ 1.54748496e+01, -1.37941206e+00, -1.74170601e+00, ...,
         -4.93775550e-01,  8.69589389e

In [51]:
data["target"][0:3,:]

array([[[103.25      ],
        [ 99.        ],
        [101.69999695]],

       [[ 99.        ],
        [101.69999695],
        [102.05000305]],

       [[101.69999695],
        [102.05000305],
        [103.05000305]]])

### data after revision with window_width_futurecast = 15min

In [59]:
data["predictor"][0:3,:]

array([[[ 1.54137332e+01, -1.52585006e+00, -5.19907009e-03, ...,
         -4.93775550e-01,  8.69589389e-01,  1.03650002e+02],
        [ 1.47829660e+01, -1.49622595e+00, -5.40706813e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  9.80500031e+01],
        [ 1.54748496e+01, -1.37941206e+00, -1.74170601e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  9.83499985e+01],
        ...,
        [ 1.70106054e+01, -1.74843705e+00, -1.92366803e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  1.02000000e+02],
        [ 1.99186356e+01, -1.78446996e+00, -2.07443309e+00, ...,
         -4.93775550e-01,  8.69589389e-01,  9.95000000e+01],
        [ 2.06585697e+01, -1.50733495e+00, -8.31857800e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  1.03250000e+02]],

       [[ 1.47829660e+01, -1.49622595e+00, -5.40706813e-01, ...,
         -4.93775550e-01,  8.69589389e-01,  9.80500031e+01],
        [ 1.54748496e+01, -1.37941206e+00, -1.74170601e+00, ...,
         -4.93775550e-01,  8.69589389e

In [60]:
data["target"][0:3,:]

array([[[ 99.        ],
        [101.69999695],
        [102.05000305]],

       [[101.69999695],
        [102.05000305],
        [103.05000305]],

       [[102.05000305],
        [103.05000305],
        [103.30000305]]])